In [29]:
import sqlite3
from sqlite3 import Error
from sqlite3 import IntegrityError
from ipyfilechooser import FileChooser
import pandas as pd
import io
import requests
import numpy
from scipy import stats
import math
from scipy.stats import ttest_ind
import ipywidgets as widgets
import tqdm

In [30]:
starting_directory = '/Volumes'
baseline_chooser = FileChooser(starting_directory)
display(baseline_chooser)

FileChooser(path='/Volumes', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

In [34]:
tap_chooser=FileChooser(starting_directory)
display(tap_chooser)

FileChooser(path='/Volumes', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

In [35]:
screens = ['PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'Neuron_Genes_Screen']

screen_chooser = widgets.Select(options=screens, value=screens[0], description='Screen:')
display(screen_chooser)

Select(description='Screen:', options=('PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'N…

In [36]:
Screen=screen_chooser.value
folder_path=baseline_chooser.selected_path
print(folder_path)

/Volumes/LavanyaLab_SSD/Screen_Data/Neuron_Genes_Screen


In [37]:
baseline_output = pd.read_csv(baseline_chooser.selected, index_col=0).drop(columns=['index'])
print(baseline_output.head())
print(baseline_output.shape)

      Time   n  Number  Instantaneous Speed  Interval Speed   Bias  \
0  490.007  44      35               0.1804          0.1032  0.829   
1  490.040  44      35               0.1776          0.1044  0.829   
2  490.070  44      35               0.1732          0.1022  0.829   
3  490.107  44      35               0.1701          0.0984  0.857   
4  490.140  44      35               0.1756          0.0983  0.829   

   Morphwidth  Midline      Area  Angular Speed  ...  Kink  Curve    Crab  \
0      0.1087   1.0640  0.140010            5.6  ...  29.2   26.6  0.0131   
1      0.1091   1.0674  0.140572            5.3  ...  29.0   26.8  0.0139   
2      0.1095   1.0694  0.140718            5.2  ...  29.3   26.8  0.0132   
3      0.1099   1.0687  0.141072            5.1  ...  29.8   26.9  0.0131   
4      0.1101   1.0706  0.141468            5.1  ...  30.9   26.5  0.0147   

   Pathlength  Plate_id      Date               Screen dataset Gene Allele  
0      14.651   A0306aa  20100310  Neur

In [38]:
tap_output = pd.read_csv(tap_chooser.selected, index_col=0)
print(tap_output.head())

      time  dura   dist      prob     speed  plate      Date Plate_id  \
0  600.026  1.93  0.366  0.954545  0.189637      1  20100516  C0512aa   
1  610.094  1.93  0.395  0.857143  0.204663      1  20100516  C0512aa   
2  620.161  1.74  0.404  0.880000  0.232184      1  20100516  C0512aa   
3  630.194  1.44  0.303  0.966667  0.210417      1  20100516  C0512aa   
4  640.261  1.40  0.281  0.818182  0.200714      1  20100516  C0512aa   

                Screen  taps         dataset     Gene  Allele  
0  Neuron_Genes_Screen   1.0  C03A3.3_ok2834  C03A3.3  ok2834  
1  Neuron_Genes_Screen   2.0  C03A3.3_ok2834  C03A3.3  ok2834  
2  Neuron_Genes_Screen   3.0  C03A3.3_ok2834  C03A3.3  ok2834  
3  Neuron_Genes_Screen   4.0  C03A3.3_ok2834  C03A3.3  ok2834  
4  Neuron_Genes_Screen   5.0  C03A3.3_ok2834  C03A3.3  ok2834  


In [39]:
PD_tap=tap_output
PD_first_tap = PD_tap[(PD_tap.taps==1)].reset_index().drop(columns="index")
PD_last_three_taps = PD_tap[((PD_tap.taps>=28)&(PD_tap.taps<=30))].reset_index().drop(columns="index")
PD_recov_taps = PD_tap[(PD_tap.taps==31)].reset_index().drop(columns="index")

print(PD_first_tap)
print("-"*55)
print(PD_last_three_taps)
print("-"*55)

print("-"*55)
PD_final_taps=PD_last_three_taps.groupby(["dataset", "Date","Plate_id","Screen","Gene","Allele","plate"]).mean().reset_index()
PD_final_taps=PD_final_taps.rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")


print("PD_final_taps")
print(PD_final_taps)
print("-"*55)


PD_first_tap=PD_first_tap.rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
PD_habit_levels = pd.merge(PD_first_tap, PD_final_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
PD_habit_levels = PD_habit_levels.dropna()

print('PD_habit_levels')
print(PD_habit_levels)
print("-"*55)
PD_habit_levels['habit_dura']=PD_habit_levels.init_dura-PD_habit_levels.final_dura
PD_habit_levels['habit_prob']=PD_habit_levels.init_prob-PD_habit_levels.final_prob
PD_habit_levels['habit_speed']=PD_habit_levels.init_speed-PD_habit_levels.final_speed
# PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y', 'Allele','Gene'])
PD_habit_levels=PD_habit_levels.drop(columns=['time_x','time_y','dist_x','dist_y'])

print('final version PD_habit_levels')
print(PD_habit_levels)
print("-"*55)

PD_recov_taps=PD_recov_taps.rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
print(PD_recov_taps)
print("-"*55)
if PD_recov_taps.empty:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='outer')
else:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')
print('After checking recov_taps empty')
print(PD_habituation)

PD_habituation = PD_habituation.drop(columns=['taps_x','taps_y'])
if Screen not in ['Neuron_Genes_Screen', 'G-Proteins_Screen']:
    PD_habituation = PD_habituation.dropna() 
print('After drop na')
print(PD_habituation)

PD_habituation['recovery_dura']=(PD_habituation.recov_dura-PD_habituation.init_dura)/PD_habituation.init_dura*100
PD_habituation['recovery_prob']=(PD_habituation.recov_prob-PD_habituation.init_prob)/PD_habituation.init_prob*100
PD_habituation['recovery_speed']=(PD_habituation.recov_speed-PD_habituation.init_speed)/PD_habituation.init_speed*100
 
if Screen in ['Neuron_Genes_Screen', 'G-Proteins_Screen']:
    tap_data=PD_habituation.dropna(subset = ['init_dura', 'init_prob', 'init_speed', 'plate', 'Date', 'Plate_id',
       'Screen', 'dataset', 'Gene', 'Allele', 'final_dura', 'final_prob',
       'final_speed', 'habit_dura', 'habit_prob', 'habit_speed'])
else:
    tap_data=PD_habituation.dropna() 


print("-"*55)
print(tap_data)

print("-"*55)


         time  dura   dist      prob     speed  plate      Date Plate_id  \
0     600.026  1.93  0.366  0.954545  0.189637      1  20100516  C0512aa   
1     600.030  2.46  0.500  0.964286  0.203252      2  20100516  C0512ab   
2     600.021  2.27  0.494  1.000000  0.217621      3  20100516  C0512ac   
3     600.006  2.47  0.639  0.954545  0.258704      4  20100516  C0512ad   
4     600.018  3.39  1.073  0.942857  0.316519      1  20100516  D0512aa   
...       ...   ...    ...       ...       ...    ...       ...      ...   
2552  600.014  2.57  0.523  0.818182  0.203502      4  20100428  C0424ad   
2553  600.023  2.80  0.666  0.974359  0.237857      1  20100423  D0419aa   
2554  600.010  2.89  0.747  0.885714  0.258478      2  20100423  D0419ab   
2555  600.028  2.55  0.651  0.888889  0.255294      3  20100423  D0419ac   
2556  600.034  2.79  0.534  0.933333  0.191398      4  20100423  D0419ad   

                   Screen  taps         dataset     Gene  Allele  
0     Neuron_Genes_S

### The below cell reads in file from online repository (github) - example in case this is possible in the future

In [10]:
# # tap_url = 'https://osf.io/du9bj/files/osfstorage/650a2f9f1e76a4230e8a99a5?raw=true'
# tap_url='https://github.com/MyYummyPancake/NRSC510B/blob/main/tap_output.csv?raw=true'
# # s=requests.get(tap_url).content
# # tap_output=pd.read_csv(io.StringIO(s.decode('utf-8')))
# tap_output=pd.read_csv(tap_url, on_bad_lines='skip', index_col=0)
# print(tap_output)

In [11]:
# print(tap_output['Gene'].unique())
# print(len(tap_output['Gene'].unique()))
# print(baseline_output['Gene'].unique())
# print(len(baseline_output['Gene'].unique()))

In [12]:
# for x in tap_output['Gene'].unique():
#     tap_output_gene=tap_output[tap_output['Gene']==x]
#     gene_tap_data=tap_output[tap_output['Date'].isin(tap_output_gene['Date'].unique())]
#     gene_tap_data_final=gene_tap_data[gene_tap_data['Gene'].isin(['N2', x])]
#     gene_tap_data_final['taps']=gene_tap_data_final['taps'].astype(int)


# for x in baseline_output['Gene'].unique():
#     baseline_output_gene=baseline_output[baseline_output['Gene']==x]
#     gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
#     gene_baseline_data_final=gene_baseline_data[gene_baseline_data['Gene'].isin(['N2', x])]
#     for a,b in zip(list_baseline_metrics, list_baseline_Tstats):
#         TTest_Allele(x, a, baseline_output, b)


# Run Statistics (T-Test and sample-mean distance) on Data

## Sample Mean Distance & Confidence Interval Calculation of Baseline Data
1. Calculate Mean

2. Calculate SEM

3. Calculate CI

4. Calculate Mean Distance From Control (Mean of Strain - Mean of Control)

In [41]:
print(tap_data.columns)

Index(['init_dura', 'init_prob', 'init_speed', 'plate', 'Date', 'Plate_id',
       'Screen', 'dataset', 'Gene', 'Allele', 'final_dura', 'final_prob',
       'final_speed', 'habit_dura', 'habit_prob', 'habit_speed', 'time',
       'recov_dura', 'dist', 'recov_prob', 'recov_speed', 'taps',
       'recovery_dura', 'recovery_prob', 'recovery_speed'],
      dtype='object')


In [42]:
print(tap_data.head())

   init_dura  init_prob  init_speed  plate      Date Plate_id  \
0       1.93   0.954545    0.189637      1  20100516  C0512aa   
1       2.46   0.964286    0.203252      2  20100516  C0512ab   
2       2.47   0.954545    0.258704      4  20100516  C0512ad   
3       3.39   0.942857    0.316519      1  20100516  D0512aa   
4       3.51   0.971429    0.272080      2  20100516  D0512ab   

                Screen         dataset     Gene  Allele  ...  habit_speed  \
0  Neuron_Genes_Screen  C03A3.3_ok2834  C03A3.3  ok2834  ...     0.048932   
1  Neuron_Genes_Screen  C03A3.3_ok2834  C03A3.3  ok2834  ...     0.027421   
2  Neuron_Genes_Screen  C03A3.3_ok2834  C03A3.3  ok2834  ...     0.069822   
3  Neuron_Genes_Screen  C03B1.5_ok2345  C03B1.5  ok2345  ...     0.102692   
4  Neuron_Genes_Screen  C03B1.5_ok2345  C03B1.5  ok2345  ...     0.100867   

   time  recov_dura  dist  recov_prob  recov_speed  taps  recovery_dura  \
0   NaN         NaN   NaN         NaN          NaN   NaN            NaN

In [10]:
def get_output_byplate(output, baseline, allele):
    drop_col_base=['Plate_id','n','Number','Time','Screen','Date','Allele']
    drop_col_taps=['Plate_id','Screen','Date','Allele','dist','plate','time',
                   'taps','recov_dura','recov_prob','recov_speed']
    drop_col = drop_col_base if baseline else drop_col_taps
    drop_col.append('Gene') if allele else drop_col.append('dataset')
     
    output_byplate = output.groupby(
        by=['Plate_id','Date','Screen','dataset','Gene','Allele'],
        as_index=False).mean().drop(columns=drop_col)
    
    return output_byplate


baseline_output_byplate=get_output_byplate(baseline_output, baseline= True ,allele=False)
print(baseline_output_byplate)
print('-'*30)
print(baseline_output_byplate.columns[1:])

print('-'*30)
tap_data_byplate=get_output_byplate(tap_data, baseline=False, allele=False)


print(tap_data_byplate)
print('-'*30)
print(tap_data_byplate.columns)
print('-'*50)

print("allele output")
print('-'*50)

baseline_output_allele_byplate=get_output_byplate(baseline_output,baseline=True, allele=True)
print(baseline_output_allele_byplate)
print('-'*50)
print(baseline_output_allele_byplate.columns)
# print(baseline_output_byplate[['Gene','Instantaneous Speed']].copy())

print('-'*50)
tap_data_allele_byplate= get_output_byplate(tap_data, baseline=False, allele=True)

print(tap_data_allele_byplate)
print('-'*50)
print(tap_data_allele_byplate.columns)

        Gene  Instantaneous Speed  Interval Speed      Bias  Morphwidth  \
0         N2             0.073405        0.085089  0.264767    0.120511   
1     egl-30             0.149717        0.138849  0.348832    0.086105   
2     egl-30             0.205273        0.151157  0.512817    0.142133   
3     egl-30             0.211926        0.174724  0.680530    0.132373   
4     egl-30             0.145660        0.074194  0.509406    0.077047   
...      ...                  ...             ...       ...         ...   
2556      N2             0.136533        0.092684  0.766364    0.096246   
2557      N2             0.144659        0.101192  0.699724    0.102023   
2558      N2             0.096483        0.080709  0.569569    0.099812   
2559      N2             0.124766        0.093121  0.666172    0.101687   
2560      N2             0.090876        0.076118  0.525435    0.103336   

       Midline      Area  Angular Speed  Aspect Ratio       Kink      Curve  \
0     1.174377  0.16

In [11]:
print(tap_data_allele_byplate[tap_data_allele_byplate.dataset=='N2_XJ1'])

     dataset  init_dura  init_prob  init_speed  final_dura  final_prob  \
230   N2_XJ1       1.85   0.898305    0.283243    0.445000    0.293836   
233   N2_XJ1       1.68   0.933333    0.288095    0.326667    0.262581   
236   N2_XJ1       1.67   0.941176    0.302395    0.326667    0.292089   
240   N2_XJ1       1.92   0.936508    0.265104    0.556667    0.453075   
246   N2_XJ1       1.90   0.893617    0.288947    0.476667    0.440753   
...      ...        ...        ...         ...         ...         ...   
2326  N2_XJ1       2.18   0.909091    0.251376    0.330000    0.400092   
2327  N2_XJ1       2.08   0.857143    0.274038    0.383333    0.349586   
2328  N2_XJ1       1.66   0.968750    0.230120    0.453333    0.396297   
2329  N2_XJ1       2.81   0.937500    0.269751    0.336667    0.287433   
2330  N2_XJ1       2.01   0.857143    0.223383    0.460000    0.388383   

      final_speed  habit_dura  habit_prob  habit_speed  recovery_dura  \
230      0.222063    1.405000    0.604

In [27]:

def extract_phenotypes(df):
    ''' 
    input: one pd.DataFrame with multiple columns (1st column is the index, the other are phenotypes)
    returns: list of pd.DataFrame, each DataFrame with 2 columns: index and 1 phenotype, 
            for how many phenotypes there are in the input
    '''
    list_phenotypes_df = []
    phenotypes = df.columns[1:]
    index = df.columns[0]
    for i in phenotypes:
        new_df = df[[index, i]].copy()
        list_phenotypes_df.append(new_df)
    return list_phenotypes_df


# baseline_output_gene=baseline_output[baseline_output['dataset']==x]
# gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
# gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2', x])]

def calculate_MSD(list_of_dfs, by):
    '''
    input: list of dataframes with 2 columns (1st one is index, 2nd one are indiviudal phenotypes)
            output from extract_phenotypes function

    returns: the same list of dataframes with additional calculated columns for:
                    1. mean
                    2. count
                    3. std
                    4. ci95_hi
                    5. ci95_low
                    6. ci95
    '''
    new_list_of_dfs=[]
    for i in list_of_dfs:
        calculate_1= i.groupby(by=[by], as_index=False).agg(['mean', 'count', 'sem'])
        calculate_2= ci95(calculate_1)
        if Screen=="Neuron_Genes_Screen":
            N2_data = calculate_2[calculate_2.index=='N2'] if by=="Gene" else calculate_2[calculate_2.index.isin(['N2_XJ1','N2_N2'])]
        else:
            N2_data = calculate_2[calculate_2.index=='N2']
        calculate_2.iloc[:,0] = calculate_2.iloc[:,0]-N2_data.iloc[0,0]
        calculate_2.iloc[:,3] = calculate_2.iloc[:,3]-N2_data.iloc[0,0]
        calculate_2.iloc[:,4] = calculate_2.iloc[:,4]-N2_data.iloc[0,0]
        new_list_of_dfs.append(calculate_2)
    return new_list_of_dfs


def ci95(df):
    """
    input: df of 4 columns: index, mean, count, std

    returns: df of 6 columns: index, mean, count, std, ci95_hi, ci95_low

    """
    for metric in df.columns.levels[0]:
        if metric == 'Gene':
            pass
        else:
            ci95_hi = []
            ci95_lo = []
            for i in df[metric].index:
                m, c, s = df[metric].loc[i]
                ci95_hi.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[1])
                ci95_lo.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[0])
            df[metric,'ci95_hi'] = ci95_hi
            df[metric,'ci95_lo'] = ci95_lo
            # df[metric,'ci95']=list(zip(ci95_lo,ci95_hi))
    return df


def get_MSD(list_MSD):
    '''
    input: List of dataframes, each representing a phenotype with calculated MSD.

    returns: Single combined dataframe joining all input dataframes with MSD values.
    '''
    for a in list_MSD:
        if a.columns.levels[0] == list_MSD[0].columns.levels[0]:
            MSD=a
        else:
            MSD=MSD.join(a)
    return MSD



In [28]:
def get_combined_MSD(baseline_byplate,tap_byplate, by=['Gene','dataset']):
    """
    Combines MSD datafram from baseline plates and tap plates

    input:
        - baseline_byplate: baseline data by plate
        - tap_byplate: tap data by plate
        - by: what to group by "Gene" or "dataset"
    returns:
        - combined MSD dataframe
    """
    list_baseline_MSD=calculate_MSD(extract_phenotypes(baseline_byplate), by=by)

    list_tap_MSD=calculate_MSD(extract_phenotypes(tap_byplate), by=by)

    baseline_MSD = get_MSD(list_baseline_MSD)
    
    tap_MSD = get_MSD(list_tap_MSD)

    combined_MSD = baseline_MSD.join(tap_MSD, on=by)

    combined_MSD=combined_MSD.rename(columns={"habit_dura":"Habituation of Response Duration",
                                         "habit_prob": "Habituation of Respones Probability",
                                         "habit_speed":"Habituation of Response Speed",
                                         "init_dura": "Initial Response Duration",
                                         "init_prob": "Initial Response Probability",
                                         "init_speed": "Initial Response Speed",
                                         "final_dura": "Final Response Duration",
                                         "final_prob": "Final Response Probability",
                                         "final_speed": "Final Response Speed",
                                         "recovery_dura": "Spontaneous Recovery of Response Duration",
                                         "recovery_prob": "Spontaneous Recovery of Response Probability",
                                         "recovery_speed": "Spontaneous Recovery of Response Speed"})

    combined_MSD=combined_MSD.reset_index()
    combined_MSD.columns = combined_MSD.columns.to_flat_index().str.join('-')
    combined_MSD=combined_MSD.rename(columns={by+"-": by})
    combined_MSD['Screen']=Screen
    
    return combined_MSD

## Gene-level SMD

In [29]:
combined_MSD=get_combined_MSD(baseline_output_byplate,
                              tap_data_byplate, 
                              by='Gene')

/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [30]:
print(combined_MSD)

        Gene  Instantaneous Speed-mean  Instantaneous Speed-count  \
0    C03A3.3                 -0.052347                          4   
1    C03B1.5                 -0.016123                          4   
2    C04B4.2                 -0.085188                          4   
3    C04E6.5                 -0.031810                          4   
4    C27B7.7                 -0.019143                          4   
..       ...                       ...                        ...   
435    wwp-1                 -0.094905                          4   
436    xol-1                 -0.054133                          4   
437   zig-12                 -0.047364                          4   
438    zig-3                 -0.069833                          4   
439    zig-7                 -0.060913                          4   

     Instantaneous Speed-sem  Instantaneous Speed-ci95_hi  \
0                   0.005000                    -0.036435   
1                   0.006724                     

## Allele-level SMD

In [31]:
print(baseline_output_allele_byplate[baseline_output_allele_byplate.dataset=='N2_XJ1'])
print(tap_data_allele_byplate[tap_data_allele_byplate.dataset=='N2_N2'])

     dataset  Instantaneous Speed  Interval Speed      Bias  Morphwidth  \
258   N2_XJ1             0.078686        0.081476  0.243865    0.107639   
262   N2_XJ1             0.135306        0.098450  0.628021    0.107281   
266   N2_XJ1             0.140936        0.097159  0.675668    0.104622   
270   N2_XJ1             0.150067        0.095419  0.723580    0.104706   
274   N2_XJ1             0.128076        0.095757  0.569462    0.096748   
...      ...                  ...             ...       ...         ...   
2556  N2_XJ1             0.136533        0.092684  0.766364    0.096246   
2557  N2_XJ1             0.144659        0.101192  0.699724    0.102023   
2558  N2_XJ1             0.096483        0.080709  0.569569    0.099812   
2559  N2_XJ1             0.124766        0.093121  0.666172    0.101687   
2560  N2_XJ1             0.090876        0.076118  0.525435    0.103336   

       Midline      Area  Angular Speed  Aspect Ratio       Kink      Curve  \
258   1.105549  0.14

In [32]:
allele_combined_MSD=get_combined_MSD(baseline_output_allele_byplate,
                                     tap_data_allele_byplate, 
                                     by='dataset')

/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [33]:
print(allele_combined_MSD)

            dataset  Instantaneous Speed-mean  Instantaneous Speed-count  \
0    C03A3.3_ok2834                 -0.049727                          4   
1    C03B1.5_ok2345                 -0.013503                          4   
2    C04B4.2_ok1212                 -0.082568                          4   
3    C04E6.5_ok1497                 -0.029190                          4   
4    C27B7.7_ok2868                 -0.016523                          4   
..              ...                       ...                        ...   
454    wwp-1_ok1102                 -0.092285                          4   
455        xol-1_y9                 -0.051513                          4   
456   zig-12_ok2994                 -0.044744                          4   
457    zig-3_ok1476                 -0.067213                          4   
458    zig-7_ok2329                 -0.058293                          4   

     Instantaneous Speed-sem  Instantaneous Speed-ci95_hi  \
0                   0.0050

## T-Stat analysis

In [34]:
def baseline_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframe and list of metrics for baseline analysis

    input:
        - by: what to group by "Gene" or "dataset"
    returns:
        - list_baseline_Tstats: to store t-statistics
        - list_baseline_metrics: to store metic names
    """
    PD_baseline_instantspeed_T=pd.DataFrame(columns = [by,"Instantaneous Speed"])
    PD_baseline_intspeed_T=pd.DataFrame(columns = [by,"Interval Speed"])
    PD_baseline_bias_T=pd.DataFrame(columns = [by,"Bias"])
    PD_baseline_morphwidth_T=pd.DataFrame(columns = [by,"Morphwidth"])
    PD_baseline_midline_T=pd.DataFrame(columns = [by,"Midline"])
    PD_baseline_area_T=pd.DataFrame(columns = [by,"Area"])
    PD_baseline_angularspeed_T=pd.DataFrame(columns = [by,"Angular Speed"])
    PD_baseline_aspectratio_T=pd.DataFrame(columns = [by,"Aspect Ratio"])
    PD_baseline_kink_T=pd.DataFrame(columns = [by,"Kink"])
    PD_baseline_curve_T=pd.DataFrame(columns = [by,"Curve"])
    PD_baseline_crab_T=pd.DataFrame(columns = [by,"Crab"])
    PD_baseline_pathlength_T=pd.DataFrame(columns = [by,"Pathlength"])

    list_baseline_Tstats=[PD_baseline_instantspeed_T,
                        PD_baseline_intspeed_T,
                        PD_baseline_bias_T,
                        PD_baseline_morphwidth_T,
                        PD_baseline_midline_T,
                        PD_baseline_area_T,
                        PD_baseline_angularspeed_T,
                        PD_baseline_aspectratio_T,
                        PD_baseline_kink_T,
                        PD_baseline_curve_T,
                        PD_baseline_crab_T,
                        PD_baseline_pathlength_T]

    list_baseline_metrics=["Instantaneous Speed",
                        "Interval Speed",
                        "Bias",
                        "Morphwidth",
                        "Midline",
                        "Area",
                        "Angular Speed",
                        "Aspect Ratio",
                        "Kink",
                        "Curve",
                        "Crab",
                        "Pathlength"]
    return list_baseline_Tstats, list_baseline_metrics

In [35]:
def tap_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframe and list of metrics for tap analysis

    input:
        - by: what to group by "Gene" or "dataset"
    returns:
        - list_baseline_Tstats: to store t-statistics
        - list_baseline_metrics: to store metic names
    """
    recovery_dura=pd.DataFrame(columns = [by,"Recovery Duration"])
    recovery_prob=pd.DataFrame(columns = [by,"Recovery Probability"])
    recovery_speed=pd.DataFrame(columns = [by,"Recovery Speed"])
    init_dura=pd.DataFrame(columns = [by,"Initial Duration"])
    init_prob=pd.DataFrame(columns = [by,"Initial Probability"])
    init_speed=pd.DataFrame(columns = [by,"Initial Speed"])
    final_dura=pd.DataFrame(columns = [by,"Final Duration"])
    final_prob=pd.DataFrame(columns = [by,"Final Probability"])
    final_speed=pd.DataFrame(columns = [by,"Final Speed"])
    hab_dura=pd.DataFrame(columns = [by,"Habituation of Duration"])
    hab_prob=pd.DataFrame(columns = [by,"Habituation of Probability"])
    hab_speed=pd.DataFrame(columns = [by,"Habituation of Speed"])

    list_tap_Tstats = [recovery_dura,
                    recovery_prob,
                    recovery_speed,
                    init_dura,
                    init_prob,
                    init_speed,
                    final_dura,
                    final_prob,
                    final_speed,
                    hab_dura,
                    hab_prob,
                    hab_speed]
    list_tap_metrics = ["recovery_dura",
                        "recovery_prob",
                        "recovery_speed",
                        "init_dura",
                        "init_prob",
                        "init_speed",
                        "final_dura",
                        "final_prob",
                        "final_speed",
                        "habit_dura",
                        "habit_prob",
                        "habit_speed"]
    return list_tap_Tstats, list_tap_metrics

In [58]:
def TTest(Type, DF_ref, output, by=["Gene", "dataset"]):
    """
    Perform two sample t-test for each unique Gene/dataset column in the Df_ref
    input: 
        - a:column name of values 
        - DF_ref:reference dataframe
        - output: output df to store results in 
        - by: what to group by "Gene" or "dataset"
    """
    for a in DF_ref[by].unique():
        Tstat_a =ttest_ind(DF_ref[DF_ref.dataset == a][Type], DF_ref[DF_ref.Allele.isin(["XJ1","N2"])][Type],equal_var=False)[0]
        Tstat_g= ttest_ind(DF_ref[DF_ref.Gene == a][Type], DF_ref[DF_ref.Gene == "N2"][Type],equal_var=False)[0]
        Tstat = Tstat_g if by=="Gene" else Tstat_a
        row=[a, Tstat]
        output.loc[len(output)]=row
    # print(output)

def do_TTest(by=["Gene", "dataset"], baseline=["true", "false"]):
    """
    Perform TTest function for each unique Gene/dataset column in baseline_output/tap_data
    
    input: 
        - by: what to group by "Gene" or "dataset"
        - baseline: whether or not to use baseline data

    returns: sorted T-statistics dataframe
    """

    if baseline=="true":
        list_Tstats,list_metrics=baseline_metrics(by)
        data = baseline_output
    else:
        list_Tstats,list_metrics=tap_metrics(by)
        data =tap_data
    for x in data[by].unique():
        if Screen=="Neuron_Genes_Screen":
            condition = x in (["N2"] if by == "Gene" else ["N2_XJ1", "N2_N2"])
        else:
            condition = (x =="N2")
        if condition:
            pass
        else:
            output_gene=data[data[by]==x]
            gene_data=data[data['Date'].isin(output_gene['Date'].unique())]
            if Screen=="Neuron_Genes_Screen":
                gene_data_final = gene_data[gene_data[by].isin(['N2', x])] if by=="Gene" else gene_data[gene_data[by].isin(['N2_N2','N2_XJ1', x])]
            else:
                gene_data_final = gene_data[gene_data[by].isin(['N2', x])]

            for a,b in zip(list_metrics, list_Tstats):
                TTest(a, gene_data_final, b, by) #calls t test function
    
    PD_Tstats=pd.DataFrame()
    for a in list_Tstats:
        b=a.groupby([by], as_index=False).mean()
        if b.columns.values[1] == list_Tstats[0].columns.values[1]:
            PD_Tstats=b
        else:
            PD_Tstats=PD_Tstats.join(b.iloc[:,1])
            
    PD_Tstats=PD_Tstats.set_index(by)
    return PD_Tstats
            

## T-stat analysis for Baseline data

In [52]:
print(baseline_output.head())

      Time   n  Number  Instantaneous Speed  Interval Speed   Bias  \
0  490.007  44      35               0.1804          0.1032  0.829   
1  490.040  44      35               0.1776          0.1044  0.829   
2  490.070  44      35               0.1732          0.1022  0.829   
3  490.107  44      35               0.1701          0.0984  0.857   
4  490.140  44      35               0.1756          0.0983  0.829   

   Morphwidth  Midline      Area  Angular Speed  ...  Kink  Curve    Crab  \
0      0.1087   1.0640  0.140010            5.6  ...  29.2   26.6  0.0131   
1      0.1091   1.0674  0.140572            5.3  ...  29.0   26.8  0.0139   
2      0.1095   1.0694  0.140718            5.2  ...  29.3   26.8  0.0132   
3      0.1099   1.0687  0.141072            5.1  ...  29.8   26.9  0.0131   
4      0.1101   1.0706  0.141468            5.1  ...  30.9   26.5  0.0147   

   Pathlength  Plate_id      Date               Screen dataset Gene Allele  
0      14.651   A0306aa  20100310  Neur

### Allele-level T-stat analysis of baseline data

In [53]:
PD_baseline_Tstats_allele = do_TTest("dataset", baseline="true") # get sorted T-statistics DataFrame 

In [54]:
PD_baseline_Tstats_allele

,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
dataset,,,,,,,,,,,,
C03A3.3_ok2834,-190.957166,-59.093280,-191.180979,-45.336975,-175.944216,-116.512954,-25.578709,74.654209,94.924872,92.264234,-42.391910,-8.012066
C03B1.5_ok2345,7.896495,52.814696,-25.494351,-3.372824,-44.948145,-17.373564,-3.274209,43.963327,23.827817,117.511393,11.048879,-98.373299
C04B4.2_ok1212,-802.684290,-225.302995,-581.538406,-211.390219,-930.813019,-682.003735,-154.303916,122.727550,155.567648,-54.324155,-321.010585,-153.185959
C04E6.5_ok1497,-222.837557,-56.077612,-147.002562,-226.912862,-325.288293,-356.825455,-53.760725,90.059869,109.589705,-13.590514,-204.869955,47.271796
C27B7.7_ok2868,-162.948777,17.108218,-168.026982,-607.866923,-1468.143139,-1295.500409,105.744052,202.041194,160.459022,55.737730,-13.479215,46.028674
...,...,...,...,...,...,...,...,...,...,...,...,...
wwp-1_ok1102,-844.053487,-308.320998,-559.510474,-534.351037,-1048.961175,-993.631155,-97.610932,195.974246,194.376830,23.593720,-358.533551,-55.949491
xol-1_y9,-264.748399,-157.835315,-222.700191,19.228221,26.174383,12.720984,-100.855157,70.230133,106.019590,112.897628,-148.048228,-36.496929
zig-12_ok2994,-252.951838,-117.113916,-283.533806,36.034517,-90.806844,-24.858545,-88.352275,89.434064,115.036622,55.519882,-107.532222,-88.079583


In [55]:
print(PD_baseline_Tstats_allele)
PD_baseline_Tstats_allele_sorted=PD_baseline_Tstats_allele.sort_index()


                Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
dataset                                                                       
C03A3.3_ok2834          -190.957166      -59.093280 -191.180979  -45.336975   
C03B1.5_ok2345             7.896495       52.814696  -25.494351   -3.372824   
C04B4.2_ok1212          -802.684290     -225.302995 -581.538406 -211.390219   
C04E6.5_ok1497          -222.837557      -56.077612 -147.002562 -226.912862   
C27B7.7_ok2868          -162.948777       17.108218 -168.026982 -607.866923   
...                             ...             ...         ...         ...   
wwp-1_ok1102            -844.053487     -308.320998 -559.510474 -534.351037   
xol-1_y9                -264.748399     -157.835315 -222.700191   19.228221   
zig-12_ok2994           -252.951838     -117.113916 -283.533806   36.034517   
zig-3_ok1476            -375.743600     -232.626497 -123.071956 -701.941777   
zig-7_ok2329            -258.933965      -99.697571 

### Gene-level T-stat analysis of baseline data

In [40]:
PD_baseline_Tstats=do_TTest("Gene", baseline="true") # get sorted T-statistics DataFrame 

In [42]:
PD_baseline_Tstats

,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
Gene,,,,,,,,,,,,
C03A3.3,-190.957166,-59.093280,-191.180979,-45.336975,-175.944216,-116.512954,-25.578709,74.654209,94.924872,92.264234,-42.391910,-8.012066
C03B1.5,7.896495,52.814696,-25.494351,-3.372824,-44.948145,-17.373564,-3.274209,43.963327,23.827817,117.511393,11.048879,-98.373299
C04B4.2,-802.684290,-225.302995,-581.538406,-211.390219,-930.813019,-682.003735,-154.303916,122.727550,155.567648,-54.324155,-321.010585,-153.185959
C04E6.5,-222.837557,-56.077612,-147.002562,-226.912862,-325.288293,-356.825455,-53.760725,90.059869,109.589705,-13.590514,-204.869955,47.271796
C27B7.7,-162.948777,17.108218,-168.026982,-607.866923,-1468.143139,-1295.500409,105.744052,202.041194,160.459022,55.737730,-13.479215,46.028674
...,...,...,...,...,...,...,...,...,...,...,...,...
wwp-1,-844.053487,-308.320998,-559.510474,-534.351037,-1048.961175,-993.631155,-97.610932,195.974246,194.376830,23.593720,-358.533551,-55.949491
xol-1,-264.748399,-157.835315,-222.700191,19.228221,26.174383,12.720984,-100.855157,70.230133,106.019590,112.897628,-148.048228,-36.496929
zig-12,-252.951838,-117.113916,-283.533806,36.034517,-90.806844,-24.858545,-88.352275,89.434064,115.036622,55.519882,-107.532222,-88.079583


In [56]:
print(PD_baseline_Tstats.head())
PD_baseline_Tstats_sorted=PD_baseline_Tstats.sort_index()


         Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
Gene                                                                   
C03A3.3          -190.957166      -59.093280 -191.180979  -45.336975   
C03B1.5             7.896495       52.814696  -25.494351   -3.372824   
C04B4.2          -802.684290     -225.302995 -581.538406 -211.390219   
C04E6.5          -222.837557      -56.077612 -147.002562 -226.912862   
C27B7.7          -162.948777       17.108218 -168.026982 -607.866923   

             Midline         Area  Angular Speed  Aspect Ratio        Kink  \
Gene                                                                         
C03A3.3  -175.944216  -116.512954     -25.578709     74.654209   94.924872   
C03B1.5   -44.948145   -17.373564      -3.274209     43.963327   23.827817   
C04B4.2  -930.813019  -682.003735    -154.303916    122.727550  155.567648   
C04E6.5  -325.288293  -356.825455     -53.760725     90.059869  109.589705   
C27B7.7 -1468.143139 -1295.

## T-stat analysis for tap-response data

In [57]:
print(tap_data.head())

   init_dura  init_prob  init_speed  plate      Date Plate_id  \
0       1.93   0.954545    0.189637      1  20100516  C0512aa   
1       2.46   0.964286    0.203252      2  20100516  C0512ab   
2       2.47   0.954545    0.258704      4  20100516  C0512ad   
3       3.39   0.942857    0.316519      1  20100516  D0512aa   
4       3.51   0.971429    0.272080      2  20100516  D0512ab   

                Screen         dataset     Gene  Allele  ...  habit_speed  \
0  Neuron_Genes_Screen  C03A3.3_ok2834  C03A3.3  ok2834  ...     0.048932   
1  Neuron_Genes_Screen  C03A3.3_ok2834  C03A3.3  ok2834  ...     0.027421   
2  Neuron_Genes_Screen  C03A3.3_ok2834  C03A3.3  ok2834  ...     0.069822   
3  Neuron_Genes_Screen  C03B1.5_ok2345  C03B1.5  ok2345  ...     0.102692   
4  Neuron_Genes_Screen  C03B1.5_ok2345  C03B1.5  ok2345  ...     0.100867   

   time  recov_dura  dist  recov_prob  recov_speed  taps  recovery_dura  \
0   NaN         NaN   NaN         NaN          NaN   NaN            NaN

### Allele level T-stat analysis of tap response data

In [59]:
PD_habituation_Tstats_allele = do_TTest("dataset", baseline="false") # get sorted T-statistics DataFrame 

/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, 

In [60]:
print(PD_habituation_Tstats_allele.head())
PD_habituation_Tstats_allele_sorted=PD_habituation_Tstats_allele.sort_index()


                Recovery Duration  Recovery Probability  Recovery Speed  \
dataset                                                                   
C03A3.3_ok2834                NaN                   NaN             NaN   
C03B1.5_ok2345                NaN                   NaN             NaN   
C04B4.2_ok1212                NaN                   NaN             NaN   
C04E6.5_ok1497                NaN                   NaN             NaN   
C27B7.7_ok2868                NaN                   NaN             NaN   

                Initial Duration  Initial Probability  Initial Speed  \
dataset                                                                
C03A3.3_ok2834          0.323943             1.203739      -1.644781   
C03B1.5_ok2345          5.059223             1.218837       2.687758   
C04B4.2_ok1212               NaN                  NaN            NaN   
C04E6.5_ok1497         31.457259             2.683535       3.985107   
C27B7.7_ok2868         -3.278810          

### Gene-level T-stat analysis of Tap response data

In [61]:
PD_habituation_Tstats = do_TTest("Gene", baseline="false") # get sorted T-statistics DataFrame 

/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, 

In [62]:
print(PD_habituation_Tstats)
PD_habituation_Tstats_sorted=PD_habituation_Tstats.sort_index()


         Recovery Duration  Recovery Probability  Recovery Speed  \
Gene                                                               
C03A3.3                NaN                   NaN             NaN   
C03B1.5                NaN                   NaN             NaN   
C04B4.2                NaN                   NaN             NaN   
C04E6.5                NaN                   NaN             NaN   
C27B7.7                NaN                   NaN             NaN   
...                    ...                   ...             ...   
wwp-1                  NaN                   NaN             NaN   
xol-1                  NaN                   NaN             NaN   
zig-12                 NaN                   NaN             NaN   
zig-3                  NaN                   NaN             NaN   
zig-7                  NaN                   NaN             NaN   

         Initial Duration  Initial Probability  Initial Speed  Final Duration  \
Gene                              

# Merging t-stat data into one dataset

In [64]:
def pop_cols(combined):
    """
    Reorders columns in the combined dataframe. 
    (pops specific columns["Area","Midline", "Morphwidth","Angular Speed"] and
    reinserts at different positions)

    input:
        combined: dataframe with columns to be reordered

    """
    first_col=combined.pop("Area")
    combined.insert(0,"Area",first_col)

    first_col=combined.pop("Midline")
    combined.insert(0,"Midline",first_col)

    first_col=combined.pop("Morphwidth")
    combined.insert(0,"Morphwidth",first_col)

    first_col=combined.pop("Angular Speed")
    combined.insert(5,"Angular Speed",first_col)

def pop_last(combined):
    """
    Reorders the last three columns of the combined dataframe.
    input:
        combined: dataframe with columns to be reordered

    """
    last_col=combined.pop("Spontaneous Recovery of Response Duration")
    combined.insert(23,"Spontaneous Recovery of Response Duration",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Probability")
    combined.insert(23,"Spontaneous Recovery of Response Probability",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Speed")
    combined.insert(23,"Spontaneous Recovery of Response Speed",last_col)

def rename_columns(df):
    '''
    Renames columns in the input dataframe
    input:
        combined: dataframe with columns to be renamed   
    returns:
        input dataframe with renamed columns 
    '''
    renames = {
        "Habituation of Duration": "Habituation of Response Duration",
        "Habituation of Probability": "Habituation of Respones Probability",
        "Habituation of Speed": "Habituation of Response Speed",
        "Initial Duration": "Initial Response Duration",
        "Initial Probability": "Initial Response Probability",
        "Initial Speed": "Initial Response Speed",
        "Final Duration": "Final Response Duration",
        "Final Probability": "Final Response Probability",
        "Final Speed": "Final Response Speed",
        "Recovery Duration": "Spontaneous Recovery of Response Duration",
        "Recovery Probability": "Spontaneous Recovery of Response Probability",
        "Recovery Speed": "Spontaneous Recovery of Response Speed"
    }
    return df.rename(columns=renames)

def merge_Tstats(baseline, habituation, by=["Gene", "dataset"]):
    """
    merge two dataframes based on the Gene/dataset
    normalize the merged dataframe and then return it with melted version

    input:
        - baseline: baseline dataframe to merge
        - habituation: habituation dataframe to merge
        - by: what to group by "Gene" or "dataset"
    """

    #merge baseline and habituation data
    combined_Tstats = pd.merge(baseline, habituation, on=[by], how='left')
    combined_Tstats = combined_Tstats.sort_index() # sort by index
    print("combined_Tstats")
    print(combined_Tstats)

    #normalise combined dataframe by subtracting mean and div by sd
    combined_Tstats_normalized = (combined_Tstats-combined_Tstats.mean())/combined_Tstats.std()

    if by=="dataset" and Screen=="Neuron_Genes_Screen":
        print(combined_Tstats_normalized[combined_Tstats_normalized.index=="N2_XJ1"])
        combined_Tstats_normalized_2 = combined_Tstats_normalized-combined_Tstats_normalized[combined_Tstats_normalized.index=="N2_XJ1"].squeeze()
        print("combined Tstats normalized N2_XJ1 data")
        print(combined_Tstats_normalized_2[combined_Tstats_normalized_2.index=="N2_XJ1"])
    else :
        print(combined_Tstats_normalized[combined_Tstats_normalized.index=="N2"])
        combined_Tstats_normalized_2 = combined_Tstats_normalized-combined_Tstats_normalized[combined_Tstats_normalized.index=="N2"].squeeze()  
        print("combined Tstats normalized N2 data")
        print(combined_Tstats_normalized_2[combined_Tstats_normalized_2.index=="N2"])


    print("combined_Tstats_normalize")
    print(combined_Tstats_normalized)

    pop_cols(combined_Tstats) #reorder columns
    print("combined_Tstats")
    print(combined_Tstats)

    #rename columns of combined and normalized df
    combined_Tstats=rename_columns(combined_Tstats)
    combined_Tstats_normalized_2=rename_columns(combined_Tstats_normalized_2)

    print("combined_Tstats_normalize_2")
    print(combined_Tstats_normalized_2)
    
    pop_cols(combined_Tstats_normalized_2) #reorder columns
    pop_last(combined_Tstats_normalized_2) #reorder columns

    print("combined_Tstats_normalize_2")
    print(combined_Tstats_normalized_2)

    # Melt the combined dataframe
    combined_Tstats_melted=combined_Tstats.reset_index()
    combined_Tstats_melted=pd.melt(combined_Tstats_melted, id_vars=[by],
                                var_name='Metric',
                                value_name='T_score')
    # Sort the melted dataframe by T_score
    combined_Tstats_melted_sorted=combined_Tstats_melted.sort_values(by=['T_score'])
    print("combined_Tstats_melted_sorted")
    print(combined_Tstats_melted_sorted)

    # Melt the normalized dataframe
    combined_Tstats_normalized_melted=combined_Tstats_normalized_2.reset_index()
    combined_Tstats_normalized_melted=pd.melt(combined_Tstats_normalized_melted, id_vars=[by],
                                                   var_name='Metric',
                                                   value_name='T_score')
    print("combined_Tstats_normalized_melted")
    print(combined_Tstats_normalized_melted)

    #add Screen column to df and its melted version
    combined_Tstats_normalized_2['Screen']=Screen
    combined_Tstats_normalized_melted['Screen']=Screen

    return combined_Tstats_normalized_2,combined_Tstats_normalized_melted

## Gene-level

In [65]:
combined_Tstats_normalize_2, combined_Tstats_normalized_melted = merge_Tstats(PD_baseline_Tstats,PD_habituation_Tstats, "Gene")


combined_Tstats
         Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
Gene                                                                   
C03A3.3          -190.957166      -59.093280 -191.180979  -45.336975   
C03B1.5             7.896495       52.814696  -25.494351   -3.372824   
C04B4.2          -802.684290     -225.302995 -581.538406 -211.390219   
C04E6.5          -222.837557      -56.077612 -147.002562 -226.912862   
C27B7.7          -162.948777       17.108218 -168.026982 -607.866923   
...                      ...             ...         ...         ...   
wwp-1            -844.053487     -308.320998 -559.510474 -534.351037   
xol-1            -264.748399     -157.835315 -222.700191   19.228221   
zig-12           -252.951838     -117.113916 -283.533806   36.034517   
zig-3            -375.743600     -232.626497 -123.071956 -701.941777   
zig-7            -258.933965      -99.697571 -215.421529  188.721797   

             Midline         Area  Angular Spee

## Allele level

In [66]:
combined_Tstats_normalize_allele_2, combined_Tstats_normalized_melted_allele = merge_Tstats(PD_baseline_Tstats_allele,PD_habituation_Tstats_allele, "dataset")

combined_Tstats
                Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
dataset                                                                       
C03A3.3_ok2834          -190.957166      -59.093280 -191.180979  -45.336975   
C03B1.5_ok2345             7.896495       52.814696  -25.494351   -3.372824   
C04B4.2_ok1212          -802.684290     -225.302995 -581.538406 -211.390219   
C04E6.5_ok1497          -222.837557      -56.077612 -147.002562 -226.912862   
C27B7.7_ok2868          -162.948777       17.108218 -168.026982 -607.866923   
...                             ...             ...         ...         ...   
wwp-1_ok1102            -844.053487     -308.320998 -559.510474 -534.351037   
xol-1_y9                -264.748399     -157.835315 -222.700191   19.228221   
zig-12_ok2994           -252.951838     -117.113916 -283.533806   36.034517   
zig-3_ok1476            -375.743600     -232.626497 -123.071956 -701.941777   
zig-7_ok2329            -258.933965 

In [67]:
print(tap_output)

        time  dura   dist      prob     speed  plate      Date Plate_id  \
0    600.026  1.93  0.366  0.954545  0.189637      1  20100516  C0512aa   
1    610.094  1.93  0.395  0.857143  0.204663      1  20100516  C0512aa   
2    620.161  1.74  0.404  0.880000  0.232184      1  20100516  C0512aa   
3    630.194  1.44  0.303  0.966667  0.210417      1  20100516  C0512aa   
4    640.261  1.40  0.281  0.818182  0.200714      1  20100516  C0512aa   
..       ...   ...    ...       ...       ...    ...       ...      ...   
115  851.289  0.98  0.109  0.410256  0.111224      4  20100423  D0419ad   
116  861.322  1.08  0.137  0.432432  0.126852      4  20100423  D0419ad   
117  871.389  1.42  0.213  0.437500  0.150000      4  20100423  D0419ad   
118  881.454  0.78  0.112  0.405405  0.143590      4  20100423  D0419ad   
119  891.488  0.89  0.127  0.411765  0.142697      4  20100423  D0419ad   

                  Screen  taps         dataset     Gene  Allele  
0    Neuron_Genes_Screen   1.0  C

In [68]:
print(combined_MSD)

        Gene  Instantaneous Speed-mean  Instantaneous Speed-count  \
0    C03A3.3                 -0.052347                          4   
1    C03B1.5                 -0.016123                          4   
2    C04B4.2                 -0.085188                          4   
3    C04E6.5                 -0.031810                          4   
4    C27B7.7                 -0.019143                          4   
..       ...                       ...                        ...   
435    wwp-1                 -0.094905                          4   
436    xol-1                 -0.054133                          4   
437   zig-12                 -0.047364                          4   
438    zig-3                 -0.069833                          4   
439    zig-7                 -0.060913                          4   

     Instantaneous Speed-sem  Instantaneous Speed-ci95_hi  \
0                   0.005000                    -0.036435   
1                   0.006724                     

## sqlite3 upload data

## A janky way to add data and update the sql 

1. Read table to pd.DataFrame
2. Add new data to pd.DataFrame
3. Replace old table with newly updated pd.DataFrame

In [69]:
# USE THIS CELL TO UPDATE ALL THE NEED TALBES (Also have baseline_output on the second line)

conn=sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/data_updated.db')
tap_output.to_sql('tap_response_data', conn, if_exists='append', index=False)
baseline_output.to_sql('tap_baseline_data', conn, if_exists='append', index=False)
combined_Tstats_normalize_2.reset_index().to_sql('tstat_gene_data', conn, if_exists='append', index=False)
combined_Tstats_normalize_allele_2.reset_index().to_sql('tstat_allele_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted.to_sql('gene_profile_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted_allele.to_sql('allele_profile_data', conn, if_exists='append', index=False)
combined_MSD.to_sql('gene_MSD', conn, if_exists='append', index=False)
allele_combined_MSD.to_sql('allele_MSD', conn, if_exists='append', index=False)
# combined_Tstats_melted_sorted.to_sql('allele_phenotype_data', conn, if_exists='replace', index=False)
print(conn.total_changes)
conn.close()


# Want to test edge cases of pd.to_sql functionality#############

7396329


## Run the following cells only after adding all Screen data to the database file 
### (this will alter the .db file so make a copy for backup before running the following)

In [ ]:
#tstat_gene_data
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
print(tables)
tstat_gene = pd.read_sql_query(f"SELECT * FROM tstat_gene_data", conn)
conn.close()
grouped = tstat_gene.groupby('Gene').agg(
        {col: 'mean' for col in tstat_gene.columns if col not in ['Gene', 'Screen']}
    ).reset_index()


def transform_to_list(row):
    screens = row.split(', ')
    return list(set(screens))

# Aggregate the Screen column into a list
grouped['Screen'] = tstat_gene.groupby('Gene')['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # Optionally, convert the Screen lists to a string representation
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
grouped['Screen']= grouped['Screen'].apply(transform_to_list)
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')

# Insert the aggregated data into the new table
grouped.to_sql("tstat_gene_data", conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [ ]:
#tstat_allele_data
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
print(tables)
tstat_gene = pd.read_sql_query(f"SELECT * FROM tstat_allele_data", conn)
conn.close()
grouped = tstat_gene.groupby('dataset').agg(
        {col: 'mean' for col in tstat_gene.columns if col not in ['dataset', 'Screen']}
    ).reset_index()


def transform_to_list(row):
    screens = row.split(', ')
    return list(set(screens))

# Aggregate the Screen column into a list
grouped['Screen'] = tstat_gene.groupby('dataset')['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # Optionally, convert the Screen lists to a string representation
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
grouped['Screen']= grouped['Screen'].apply(transform_to_list)
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')

# Insert the aggregated data into the new table
grouped.to_sql("tstat_allele_data", conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [61]:
#gene_profile_data
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
print(tables)
tstat_gene = pd.read_sql_query(f"SELECT * FROM gene_profile_data", conn)
conn.close()
print(tstat_gene)
grouped = tstat_gene.groupby(['Gene','Metric']).agg(
        {col: 'mean' for col in tstat_gene.columns if col not in ['Gene', 'Screen','Metric']}
    ).reset_index()
grouped['Screen'] = tstat_gene.groupby(['Gene','Metric'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)
print(grouped)
def transform_to_list(row):
    screens = row.split(', ')
    return list(set(screens))

# Aggregate the Screen column into a list
grouped['Screen'] = tstat_gene.groupby(['Gene','Metric'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # Optionally, convert the Screen lists to a string representation
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
grouped['Screen']= grouped['Screen'].apply(transform_to_list)
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')

# Insert the aggregated data into the new table
grouped.to_sql("gene_profile_data", conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

['tap_response_data', 'gene_profile_data', 'allele_profile_data', 'gene_MSD', 'allele_MSD', 'tap_baseline_data', 'tstat_gene_data', 'tstat_allele_data']
          Gene                         Metric   T_score             Screen
0     C29E4.10                     Morphwidth -0.641065          PD_Screen
1     F32D8.13                     Morphwidth  0.769580          PD_Screen
2      F35C8.1                     Morphwidth  0.056716          PD_Screen
3      F53B2.5                     Morphwidth -0.299505          PD_Screen
4     F54B11.5                     Morphwidth -0.825061          PD_Screen
...        ...                            ...       ...                ...
9835    unc-13  Habituation of Response Speed       NaN  G-Proteins_Screen
9836     unc-2  Habituation of Response Speed       NaN  G-Proteins_Screen
9837    unc-73  Habituation of Response Speed       NaN  G-Proteins_Screen
9838   unknown  Habituation of Response Speed       NaN  G-Proteins_Screen
9839    zig-12  Habitu

In [ ]:
#allele_profile_data
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
print(tables)
tstat_gene = pd.read_sql_query(f"SELECT * FROM allele_profile_data", conn)
conn.close()
print(tstat_gene)
grouped = tstat_gene.groupby(['dataset','Metric']).agg(
        {col: 'mean' for col in tstat_gene.columns if col not in ['dataset', 'Screen','Metric']}
    ).reset_index()
grouped['Screen'] = tstat_gene.groupby(['dataset','Metric'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)
print(grouped)
def transform_to_list(row):
    screens = row.split(', ')
    return list(set(screens))

# Aggregate the Screen column into a list
grouped['Screen'] = tstat_gene.groupby(['dataset','Metric'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # Optionally, convert the Screen lists to a string representation
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
grouped['Screen']= grouped['Screen'].apply(transform_to_list)
grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')

# Insert the aggregated data into the new table
grouped.to_sql("allele_profile_data", conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
#allele_MSD
# conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
# tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
# tables = pd.read_sql_query(tables_query, conn)['name'].tolist()
# print(tables)
# tstat_gene = pd.read_sql_query(f"SELECT * FROM allele_MSD", conn)
# conn.close()
# print(tstat_gene)
# def weighted_avg(df, col_mean, col_count):
#     return (df[col_mean] * df[col_count]).sum() / df[col_count].sum()

# grouped = tstat_gene.groupby('dataset').agg(
#     {col: lambda x: weighted_avg(x, col, col.replace('-mean', '-count')) 
#      for col in tstat_gene.columns if col.endswith('-mean')}
# ).reset_index()

# count_cols = [col for col in tstat_gene.columns if col.endswith('-count')]
# for col in count_cols:
#     grouped[col] = tstat_gene.groupby('dataset')[col].sum().values

# grouped['Screen'] = tstat_gene.groupby(['dataset'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)
# print(grouped)

# def compute_ci95(df, metric):
#     ci95_hi = []
#     ci95_lo = []
#     for i in df.index:
#         m = df[f'{metric}-mean'].loc[i]
#         c = df[f'{metric}-count'].loc[i]
#         s = df[f'{metric}-sem'].loc[i]
#         ci_hi, ci_lo = stats.t.interval(0.95, c-1, loc=m, scale=s)
#         ci95_hi.append(ci_hi)
#         ci95_lo.append(ci_lo)
#     df[f'{metric}-ci95_hi'] = ci95_hi
#     df[f'{metric}-ci95_lo'] = ci95_lo
#     return df

# # def transform_to_list(row):
# #     screens = row.split(', ')
# #     return list(set(screens))

# # # Aggregate the Screen column into a list
# # grouped['Screen'] = tstat_gene.groupby(['dataset'])['Screen'].apply(lambda x: list(set(x))).reset_index(drop=True)

# # # Optionally, convert the Screen lists to a string representation
# # grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
# # grouped['Screen']= grouped['Screen'].apply(transform_to_list)
# # grouped['Screen'] = grouped['Screen'].apply(lambda x: ', '.join(x))
# # conn = sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/test.db')
# metrics = set([col.split('-')[0] for col in tstat_gene.columns if col.endswith('-mean')])
# for metric in metrics:
#     grouped = compute_ci95(grouped, metric)

# print(grouped)
# # Insert the aggregated data into the new table
# # grouped.to_sql("allele_MSD", conn, if_exists='replace', index=False)

# # Commit the changes and close the connection
# conn.commit()
# conn.close()

In [60]:
grouped.Screen.unique()

array(['ASD_Screen', 'PD_Screen',
       'ASD_Screen, G-Proteins_Screen, PD_Screen, Glia_Genes_Screen',
       'G-Proteins_Screen', 'Glia_Genes_Screen', 'PD_Screen, ASD_Screen',
       'G-Proteins_Screen, ASD_Screen', 'G-Proteins_Screen, PD_Screen',
       'G-Proteins_Screen, Glia_Genes_Screen'], dtype=object)

In [47]:
tstat_gene

,Gene,Metric,T_score,Screen
0,F53B2.5,Morphwidth,-0.195640,PD_Screen
1,N2,Morphwidth,0.000000,PD_Screen
2,cat-1,Morphwidth,-0.526426,PD_Screen
3,cat-2,Morphwidth,0.498182,PD_Screen
4,catp-7,Morphwidth,-0.682773,PD_Screen
...,...,...,...,...
499,pdr-1,Habituation of Response Speed,0.012246,PD_Screen
500,rme-8,Habituation of Response Speed,0.868745,PD_Screen
501,sipa-1,Habituation of Response Speed,-1.529130,PD_Screen
502,snb-5,Habituation of Response Speed,0.359162,PD_Screen


In [ ]:
# cursor = conn.cursor()

# cursor.execute("SELECT COUNT(*) FROM table_name WHERE primary_key_column = ?", (value,))
# exists = cursor.fetchone()[0] > 0

# if exists:
#     print("Primary key already exists in the table.")
# else:
#     print("Primary key does not exist in the table.")

In [ ]:
# conn.close()

## Done!